<a href="https://colab.research.google.com/github/cardstdani/practica-eda/blob/main/NotebookEDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Práctica 1**

In [ ]:
!nvidia-smi

In [ ]:
!pip install pyvis

In [ ]:
import random
import time
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
import matplotlib.pyplot as plt
import concurrent.futures
from pyvis.network import Network
from functools import reduce
from google.colab.patches import cv2_imshow
from mpl_toolkits.mplot3d import Axes3D
import itertools

In [ ]:
class CeldaSimple:
    def __init__(self):
        self.grid = None
        self.visited = None
        self.cortocircuito = False
        self.G = Network(notebook=True)
        self.G.force_atlas_2based(gravity=-50, central_gravity=0.01, spring_length=100, spring_strength=0.08, damping=0.4, overlap=0)
        self.iteration = 0

    def Inicializar(self, n):
        self.grid = [[False] * n for _ in range(n)]
        self.visited = [[False] * n for _ in range(n)]
        self.cortocircuito = False

    def RayoCosmico(self, i, j):
        if not self.grid[i][j]:
            self.grid[i][j] = True

            self.visited = [[False] * len(self.grid) for _ in range(len(self.grid))]
            self.cortocircuito = self.helper(i, j, 0) and self.helper(i, j, 1)

    def helper(self, i, j, up, depth=0, parent=None):
        self.visited[i][j] = True
        node_name = f"{i}_{j}_{up}"
        color_intensity = int((depth / len(self.grid)) * 255)
        color = "#{:02x}{:02x}{:02x}".format(100+color_intensity, 120, 0)

        #self.G.add_node(node_name, label=f"helper({i}, {j}, {up})", color=color)
        #if parent is not None:
        #    self.G.add_edge(parent, node_name)
        if up == 0:
            if i == len(self.grid) - 1: return True
            nei = [
                [i, max(0, j - 1)],
                [i, min(len(self.grid[0]) - 1, j + 1)],
                [min(len(self.grid) - 1, i + 1), j],
                [min(len(self.grid) - 1, i + 1), min(len(self.grid[0]) - 1, j + 1)],
                [min(len(self.grid) - 1, i + 1), max(0, j - 1)],
            ]
        else:
            if i == 0: return True
            nei = [
                [i, max(0, j - 1)],
                [i, min(len(self.grid[0]) - 1, j + 1)],
                [max(0, i - 1), j],
                [max(0, i - 1), min(len(self.grid[0]) - 1, j + 1)],
                [max(0, i - 1), max(0, j - 1)],
            ]

        # Recursion
        for k in nei:
            if not self.visited[k[0]][k[1]] and self.grid[k[0]][k[1]]:
                if self.helper(k[0], k[1], up, depth + 1, node_name):
                    return True
        return False

    def visualize(self):
        self.G.show(f"Graph{self.iteration}.html")
        #self.iteration += 1
        #self.G = Network(notebook=True)
        #self.G.force_atlas_2based(gravity=-50, central_gravity=0.01, spring_length=100, spring_strength=0.08, damping=0.4, overlap=0)

    def Cortocircuito(self):
        return self.cortocircuito

    def __str__(self):
        out = ""
        for i in range(len(self.grid)):
            for j in range(len(self.grid[0])):
                out += "X" if self.grid[i][j] else "."
                if j < len(self.grid[0]) - 1:
                    out += " "
            out += "\n"
        return out

SEMILLA = 42
if __name__ == "__main__":
    #random.seed(SEMILLA)
    print("¿Validación? [S/N] ", end="")
    validar = input().strip().upper().startswith("S")
    print("Tamaño (n) = ", end="")
    n = int(input())
    num_rep = 1
    if not validar:
        print("Repeticiones = ", end="")
        num_rep = int(input())

    celda = CeldaSimple()
    tpo_total = 0
    num_rayos = 0

    for k in range(num_rep):
        # Simulación
        celda.Inicializar(n)
        num_rayos = 0
        tpo1 = time.perf_counter()
        while not celda.Cortocircuito():
            # Elegir átomo al azar y transmutarlo
            celda.RayoCosmico(random.randint(0, n - 1), random.randint(0, n - 1))
            celda.visualize()
            num_rayos += 1
        print(celda.grid*1)
        tpo2 = time.perf_counter()
        tpo_total += tpo2 - tpo1
        print("#", end="")

    print("\nTiempo medio = {:.5f}".format(tpo_total / num_rep))

    if validar:
        print("Número de rayos =", num_rayos)
        print(celda)

**Simulación específica**

In [ ]:
celda = CeldaSimple()
y = []
n = 7

random.seed()
for k in range(10000):
    celda.Inicializar(n)
    num_rayos = 0
    while not celda.Cortocircuito():
        # Elegir átomo al azar y transmutarlo
        celda.RayoCosmico(random.randint(0, n - 1), random.randint(0, n - 1))
        num_rayos += 1
    #print(num_rayos)
    y.append(num_rayos)

plt.hist(y, edgecolor='black')
plt.title('Distribution of num_rayos')
plt.xlabel('num_rayos')
plt.ylabel('Frequency')
plt.show()
print(sorted(y)[-10:])

In [ ]:
def convolution(matrix, kernel):
    mat_height, mat_width = matrix.shape
    ker_height, ker_width = kernel.shape

    # Calculate padding dimensions
    pad_height = ker_height // 2
    pad_width = ker_width // 2

    padded_matrix = np.pad(matrix, ((pad_height, pad_height), (pad_width, pad_width)), mode='constant')
    output = np.zeros_like(matrix)
    for i in range(mat_height):
        for j in range(mat_width):
            region = padded_matrix[i:i+ker_height, j:j+ker_width]
            output[i, j] = np.sum(region * kernel)

    return output

def visualize_fft(matrix):
    f_transform = np.fft.fft2(matrix)
    f_shift = np.fft.fftshift(f_transform)
    magnitude_spectrum = np.log(np.abs(f_shift) + 1)
    freq = np.abs(f_transform)

    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(14, 6))
    ax[0,0].hist(freq.ravel(), bins=100)
    ax[0,0].set_title('hist(freq)')
    ax[0,1].hist(np.log(freq).ravel(), bins=100)
    ax[0,1].set_title('hist(log(freq))')
    ax[1,0].imshow(np.log(freq), interpolation="none", cmap="gray")
    ax[1,0].set_title('log(freq)')
    ax[1,1].imshow(matrix, interpolation="none", cmap="gray")
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.imshow(magnitude_spectrum, cmap='gray')
    plt.colorbar(label='Log Magnitude')
    plt.title('2D FFT Magnitude Spectrum')
    plt.show()
    return magnitude_spectrum

def plotMatrix3D(matrix):
    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(111, projection='3d')
    x = np.arange(matrix.shape[1])
    y = np.arange(matrix.shape[0])
    x, y = np.meshgrid(x, y)
    z = matrix
    surf = ax.plot_surface(x, y, z, cmap='gray')
    fig.colorbar(surf)
    plt.title(':()')
    plt.show()

def get_embedding(matrix):
    matrix_3channel = np.stack([matrix] * 3, axis=-1) * 255

    matrix_resized = tf.image.resize(matrix_3channel, [224, 224])
    matrix_resized = np.array(matrix_resized)
    matrix_preprocessed = preprocess_input(matrix_resized)
    model = VGG16(include_top=False, weights='imagenet', pooling='max')
    embedding = model.predict(np.expand_dims(matrix_preprocessed, axis=0))
    return embedding[0]

def generate_combinations(n, n_prime):
    """Generate all possible matrices with n_prime black elements in an nxn matrix."""
    positions = list(itertools.combinations(range(n * n), n_prime))
    matrices = []
    for pos in positions:
        matrix = np.zeros(n * n)
        for p in pos:
            matrix[p] = 1
        matrices.append(matrix.reshape(n, n))
    return matrices

def plot_embeddings_3D(n, n_prime):
    # Generate combinations
    matrices = generate_combinations(n, n_prime)

    # Get embeddings for each matrix
    embeddings = [get_embedding(matrix) for matrix in matrices]

    # Extract the first 3 dimensions of each embedding for 3D plotting
    x = [emb[0] for emb in embeddings]
    y = [emb[1] for emb in embeddings]
    z = [emb[2] for emb in embeddings]

    # Plot embeddings in 3D space
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x, y, z)

    ax.set_xlabel('X Dimension')
    ax.set_ylabel('Y Dimension')
    ax.set_zlabel('Z Dimension')
    ax.set_title(f'3D Embedding of {n_prime} Black Elements in {n}x{n} Matrix')

    plt.show()

matrix = np.array(celda.grid, dtype=np.int32)
kernel = np.array([[-1, -1, -1],
                  [-1, 2, -1],
                  [-1, -1, -1]])
kernel = np.array([[0, 1, 0],
                  [0, 1, 0],
                  [0, 1, 0]])
kernel = np.array([[1, 1, 1],
                  [1, 0, 1],
                  [1, 1, 1]])/2
convolve = lambda matrix, kernel, n: reduce(lambda m, _: convolution(m, kernel), range(n), matrix)

print(matrix,convolve(matrix, kernel, 1), get_embedding(matrix))
visualize_fft(matrix)
plt.imshow(matrix, cmap='gray')
plt.show()

plt.figure(figsize=(20, 5))
for idx, i in enumerate(range(1, 20), 1):
    plt.subplot(1, 20, idx)
    plt.imshow(convolve(matrix, kernel, i), cmap='gray')
    plt.title(f"{i}")
plt.tight_layout()
plt.show()

plotMatrix3D(convolve(matrix, kernel, 10))
plot_embeddings_3D(3, 2)

In [ ]:
from itertools import combinations
from collections import deque

# Tamaño de la malla
n = 5
# Número de celdas a colorear
n_prime = 5

# Generar todas las posiciones posibles en la malla
positions = [(i, j) for i in range(n) for j in range(n)]

# Función para verificar si existe un camino desde arriba hasta abajo
def has_path(grid, start):
    """
    Utiliza DFS para verificar si existe un camino desde la fila superior
    hasta la fila inferior en la malla dada.
    """
    stack = deque([start])
    visited = set([start])

    while stack:
        i, j = stack.pop()
        if i == n - 1:
            return True

        for x, y in [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]:
            if (0 <= x < n and 0 <= y < n and
                grid[x][y] == 1 and (x, y) not in visited):
                stack.append((x, y))
                visited.add((x, y))
    return False

# Contadores para las combinaciones totales y las combinaciones con un camino
total_combinations = 0
combinations_with_path = 0
for black_cells in combinations(positions, n_prime):
    grid = [[0] * n for _ in range(n)]

    # Colorear las celdas seleccionadas
    for i, j in black_cells:
        grid[i][j] = 1

    # Verificar si existe un camino para cada celda coloreada en la fila superior
    if any(has_path(grid, (0, j)) for j in range(n) if grid[0][j] == 1):
        combinations_with_path += 1
    total_combinations += 1
print(total_combinations, combinations_with_path, combinations_with_path / total_combinations)

**Medidas Tiempo**

In [ ]:
with open('t_values_python.txt', 'w') as file:
  celda = CeldaSimple()

  for i in range(2, 40):
      for j in range(15):
          celda.Inicializar(i)
          t = time.perf_counter()
          while not celda.Cortocircuito():
              celda.RayoCosmico(random.randint(0, i - 1), random.randint(0, i - 1))
          t = time.perf_counter() - t
          print(i, j, t)
          file.write(f"{i} {t}\n")

In [ ]:
import numpy as np
from numba import njit, prange
import random
import time
import concurrent.futures

class CeldaSimple:
    def __init__(self):
        self.grid = None
        self.visited = None
        self.iterations = 0

    def Inicializar(self, n):
        self.grid = np.full((n, n), False, dtype=np.bool_)
        self.visited = np.full((n, n), False, dtype=np.bool_)

    def RayoCosmico(self, i, j):
        if not self.grid[i][j]:
            self.grid[i][j] = True
        self.iterations += 1

    def helper(self, i, j):
        return _helper_numba(self.grid, self.visited, i, j)

    def Cortocircuito(self):
        return _cortocircuito_numba(self.grid, self.visited)

    def __str__(self):
        out = ""
        for i in range(self.grid.shape[0]):
            for j in range(self.grid.shape[1]):
                out += "X" if self.grid[i][j] else "."
                if j < self.grid.shape[1] - 1:
                    out += " "
            out += "\n"
        return out

@njit
def _helper_numba(grid, visited, i, j):
    if i == grid.shape[0] - 1:
        return True
    visited[i][j] = True
    nei = [
        (max(0, i - 1), j), (min(grid.shape[0] - 1, i + 1), j),
        (i, max(0, j - 1)), (i, min(grid.shape[1] - 1, j + 1)),
        (max(0, i - 1), max(0, j - 1)),
        (min(grid.shape[0] - 1, i + 1), min(grid.shape[1] - 1, j + 1)),
        (max(0, i - 1), min(grid.shape[1] - 1, j + 1)),
        (min(grid.shape[0] - 1, i + 1), max(0, j - 1))
    ]

    for k in nei:
        if not visited[k[0]][k[1]] and grid[k[0]][k[1]]:
            if _helper_numba(grid, visited, k[0], k[1]):
                return True
    return False

@njit(parallel=True)
def _cortocircuito_numba(grid, visited):
    for i in prange(grid.shape[0]):
        for j in prange(grid.shape[1]):
            visited[i][j] = False

    for i in prange(grid.shape[1]):
        if grid[0][i]:
            if _helper_numba(grid, visited, 0, i):
                return True
    return False

def simulation_run(params):
    i, j = params
    celda = CeldaSimple()
    celda.Inicializar(i)
    t = time.perf_counter()
    while not celda.Cortocircuito():
        celda.RayoCosmico(random.randint(0, i - 1), random.randint(0, i - 1))
    t = time.perf_counter() - t
    print(i, j, t)
    return f"{i} {t}\n"

with open('t_values_python.txt', 'w') as file:
    with concurrent.futures.ThreadPoolExecutor() as executor:
        params = [(i, j) for i in range(2, 40) for j in range(15)]
        results = executor.map(simulation_run, params)

        file.writelines(results)